In [1]:
import requests
import re
import random
from bs4 import BeautifulSoup
from tqdm.auto import tqdm, trange

# Mordovians (erzya)

In [2]:
url = 'https://www.mordovians.ru/erzyanskiy_yazyk'
print(url)

https://www.mordovians.ru/erzyanskiy_yazyk


In [3]:
soup = BeautifulSoup(requests.get(url).text)

In [4]:
with open('other_sources/Эрзянский язык _ Мордовская Диаспора.html', 'r') as f:
    soup = BeautifulSoup(f.read())

In [5]:
rus = []
translations = []

for t in soup.find_all('div', {'class': 't649__text'}):
    for item in t.find('div'):
        if item.name == 'strong':
            rus.append(item.text)
            translations.append([])
        elif isinstance(item, str):
            text = item.strip().lstrip('–').lstrip('-').strip()
            if text:
                translations[-1].append(text)

In [6]:
len(rus)

1544

In [7]:
len(translations)

1544

In [8]:
rus_dedup = []
tran_dedup = []

for r, t in zip(rus, translations):
    if not r or not t:
        continue
    rus_array = [text.strip() for text in r.split(',')]
    if r == '0,9':
        rus_array = [r]
    #if len(rus_array) > 1:
    #    print(r)
    myv_array = [text.strip() for text in t[0].split(',')]
    for r2 in rus_array:
        for m2 in myv_array:
            rus_dedup.append(r2)
            tran_dedup.append(m2)

In [9]:
import pandas as pd

In [10]:
df = pd.DataFrame({'rus': rus_dedup, 'myv': tran_dedup})
df.shape

(1847, 2)

In [11]:
df.to_csv('russian-erzya-dictionary_mordovians.tsv', sep='\t')

In [78]:
df.sample(10)

,rus,myv
1015,напоить,симдемс
682,лыко,ленге
180,выводить,ливтемс
1406,старый,ташто
1645,церковь,знома кудо
1103,приехать,ардомс
1286,привязать,сюлмамс
285,удержать,кирдемс
937,пахнуть,качадомс
1270,сватать,чиямо


In [79]:
mordovians_pairs = df[['myv', 'rus']].values.tolist()

# layazyk

In [81]:
url = 'http://lazalyk.narod.ru/business.html'
print(url)

http://lazalyk.narod.ru/business.html


In [82]:
soup = BeautifulSoup(requests.get(url).text)

In [83]:
rows = [text.strip() for text in soup.find('table').text.split('\r')[3:-1]]

In [84]:
pairs = []

for row in rows:
    row = re.sub('\s*\(.*\)\s*', ' ', row).strip().split('\n')[0]
    if row.startswith('-') or row.startswith('-') or not row:
        continue
    if any(s in row for s in '.)('):
        continue
    parts = row.split(' - ')
    if len(parts) == 2:
        lhs = parts[0]
        rhs = parts[1]
    else:
        parts = row.split('-')
        if len(parts) == 2:
            lhs = parts[0]
            rhs = parts[1]
        else:
            continue
    for ru in lhs.split(','):
        for myv in rhs.split(','):
            pairs.append([myv, ru])

In [85]:
len(pairs)

1311

In [86]:
for i in range(10):
    print(random.choice(pairs))

['илештямс', 'залупить']
['эряза', 'активный']
['уштом', 'печь']
['вардо', 'батрак']
['тарка', 'место']
[' пинксэмс', 'сковать']
['кизэ', 'лето']
['кивкал', 'голец']
['лачо', 'клин']
[' валкскудо', 'банк']


In [87]:
pairs_layazyk = pairs

# mordvarf

In [88]:
url = 'https://mordvarf.com/русско-эрзянский-словарь/'
print(url)

https://mordvarf.com/русско-эрзянский-словарь/


In [89]:
soup = BeautifulSoup(requests.get(url).text)

In [90]:
pairs = []

In [91]:
def split2(text, seps=';,/\\'):
    parts = [text]
    for sep in seps:
        parts = [text.strip() for p in parts for text in p.split(sep)]
    return [p for p in parts if p]

In [92]:
div = soup.find('div', {'class': 'entry-content'})
for p in list(div.children)[:10]:
    if isinstance(p, str):
        continue
    if not p.text.strip():
        continue
    prev = None
    for c in p.children:
        if isinstance(c, str):
            if c == '\n':
                prev = None
                continue
            if prev is not None:
                for tran in split2(c):
                    for orig in split2(prev):
                        pairs.append([tran.strip(), orig.strip()])
                prev = None
                continue
        elif c.name == 'b':
            prev = c.text
        else:
            prev = None

In [93]:
print(len(pairs))

328


In [94]:
random.choice(pairs)

['чиемс', 'бежать']

In [95]:
pairs_mordvarf = pairs

# marlamuter

In [96]:
page = 746
url = 'http://marlamuter.com/muter/Эрзя/{}'.format(page)
print(url)

http://marlamuter.com/muter/Эрзя/746


In [97]:
marlamuter_pairs = []

In [98]:
for page in trange(1, 749):
    url = 'http://marlamuter.com/muter/Эрзя/{}'.format(page)
    soup = BeautifulSoup(requests.get(url).text)
    for d in soup.findAll('div', {'class': 'ik_mut'}):
        for bad_class in ['arab_bracket', 'oypeleshtysh', 'abbr']:
            for noise in d.findAll('span', {'class': bad_class}):
                noise.extract()

        for p in d.findAll('p'):
            if p.find('span', {'class': 'tanastare'}):
                p.extract()
            elif p.find('span', {'class': 'mutlan1'}) and p.find('span', {'class': 'mutlan3'}):
                marlamuter_pairs.append([
                    p.find('span', {'class': 'mutlan1'}).text.strip('.').strip().strip(';'), 
                    p.find('span', {'class': 'mutlan3'}).text.strip('.').strip().strip(';')
                ])
                p.extract()

        myv = d.find('div', {'id': 'head'}).text.strip().strip('.').strip().strip(';')
        rus = d.find('div', {'id': 'content'}).text.strip().strip('.').strip().strip(';')
        for tran in split2(myv):
            for orig in split2(rus):
                marlamuter_pairs.append([tran.strip(), orig.strip()])

  0%|          | 0/748 [00:00<?, ?it/s]

In [99]:
print(len(marlamuter_pairs))

25997


In [100]:
print(len([p for p in marlamuter_pairs if '\n' not in p[1]]))

25245


In [101]:
for _ in range(10):
    print(random.choice(marlamuter_pairs))

['эстрада', 'эстрада']
['томсезь', 'утомлённый']
['поревтемс', 'заставить загрызть']
['скалонь сывель', 'говядина']
['кудо ютконь мелявкст', 'заботы о доме']
['нуема', 'жатва']
['нолаштнемс', 'скользить']
['пси', 'жарко']
['лондадозь', 'провалившийся']
['икелевгак', 'роботамо кияк']


# Словарь Рябова

http://openedo.mrsu.ru/pluginfile.php/137136/mod_resource/content/1/Эрзянский%20язык.pdf

In [102]:
import pandas as pd

In [103]:
rya_raw = pd.read_csv('books_parallel/ryabov-tables-raw.tsv', sep='\t', header=None)

In [104]:
rya_raw

,0,1,2,3,4,5
0,NaN,анды,NaN,NaN,завтра,NaN
1,NaN,астовомс,NaN,NaN,встретиться,NaN
2,NaN,астома,NaN,NaN,встреча,NaN
3,NaN,еемс,NaN,NaN,увидеть,NaN
4,NaN,тпустить,NaN,NaN,нолдамс,NaN
...,...,...,...,...,...,...
503,Мокшэрзятнень пек кеметь обуцяст\rды коест.,У мордвы очень крепкие обычаи и\rтрадиции.,NaN,NaN,NaN,NaN
504,"Эрзятне кастыть сюро, ракшат,\rкундсить калт, ...","Эрзя выращивают хлеб, животных, ловят\rрыбу, с...",NaN,NaN,NaN,NaN
505,Эрзятнень мазыйть оршамост ды\rтантейть ярсамо...,У эрзи красивые наряды и вкусная еда.,NaN,NaN,NaN,NaN
506,"Покш чистэ аватне панить прякат,\rтеить човов ...","На большие праздники женщины пекут\rпироги, го...",NaN,NaN,NaN,NaN


In [105]:
rya_pairs = []
for i, row in rya_raw.iterrows():
    lst = row.dropna().tolist()
    if len(lst) != 2:
        continue
    er, ru = lst
    er = er.replace('\r', ' ')
    ru = ru.replace('\r', ' ')
    if len(er.split()) == 1 and er == er.lower() and any(c in ru for c in ',;/'):
        for ru2 in split2(ru):
            rya_pairs.append([er, ru2])
    else:
        rya_pairs.append([er, ru])
        
print(len(rya_pairs))

588


# Merge with a dictionary from pdf

In [110]:
from ast import literal_eval

In [111]:
dict1 = pd.read_csv('russian-moksha-erzya-dictionary_fixed.tsv', sep='\t')
dict1['mo'] = dict1['mo'].apply(literal_eval)
dict1['er'] = dict1['er'].apply(literal_eval)

In [113]:
dict1_pairs = []
for row in dict1.itertuples():
    for er in row.er:
        dict1_pairs.append((er, row.ru_fix))
len(dict1_pairs)

22499

# merge

In [114]:
joint_dict = []

for k, v in {
    'mordovians': mordovians_pairs,
    'layazyk': pairs_layazyk,
    'mordvarf': pairs_mordvarf,
    'marlamuter': [p for p in marlamuter_pairs if '\n' not in p[1] and 'Тӱҥ' not in p[1] and 'Калыкмут' not in p[1]],
    'ryabov': rya_pairs,
    'schankina': dict1_pairs,
}.items():
    df = pd.DataFrame(v)
    df.columns = ['myv', 'ru']
    df['source'] = k
    joint_dict.append(df)
joint_dict = pd.concat(joint_dict, ignore_index=True)

In [115]:
print(joint_dict.shape)

(51383, 3)


In [116]:
joint_dict.sample(10)

,myv,ru,source
46352,керявкс молемс/аштемс каршо,разрез,schankina
44688,парсте прянь ветямо,порядочность,schankina
31207,совамс,вступить,schankina
37990,кундо,крышка,schankina
50212,каямс,устранить,schankina
29992,каштан,важный,schankina
35258,пичкамс,зарасти,schankina
18274,пекше,липа,marlamuter
17606,орголемс,убегать,marlamuter
18048,парямс,париться,marlamuter


In [117]:
joint_dict.groupby('source').sample(3)

,myv,ru,source
2797,туемс,пойти,layazyk
2883,лей,река,layazyk
2993,сырькай,старуха,layazyk
5511,бензинэнь бак,бензиновый бак,marlamuter
20728,самодур,самодур,marlamuter
25511,хлоп!,хлоп!,marlamuter
1290,течи,сегодня,mordovians
992,ушо,погода,mordovians
330,качамо,дым,mordovians
3380,цярахман,град,mordvarf


In [118]:
joint_dict.to_csv('other_sources/joint_phrasebook.tsv', index=None, sep='\t')